In [22]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

/home/matheuspds/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
pd_volume_train = pd.read_csv('processed_train_volume2.csv')

In [4]:
pd_volume_train.head()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type,weekday,weekend,hour,date,holiday,time_window
0,2016-09-19 23:09:25,2,0,1,0,0,1,0,23,2016-09-19,False,"[23:00:00,23:20:00)"
1,2016-09-19 23:11:53,2,0,1,0,0,1,0,23,2016-09-19,False,"[23:00:00,23:20:00)"
2,2016-09-19 23:13:54,2,0,1,0,0,1,0,23,2016-09-19,False,"[23:00:00,23:20:00)"
3,2016-09-19 23:17:48,1,0,1,1,0,1,0,23,2016-09-19,False,"[23:00:00,23:20:00)"
4,2016-09-19 23:16:07,2,0,1,0,0,1,0,23,2016-09-19,False,"[23:00:00,23:20:00)"


In [5]:
pd_volume_train['time'] =  pd.to_datetime(pd_volume_train['time'] , format='%Y-%m-%d %H:%M:%S')
#pd_volume_train = pd_volume_train.set_index(['time_window'])

# 车流量
pd_volume_train = pd_volume_train.groupby([pd.Grouper(freq='20T', key='time'), 'tollgate_id', 'direction', 'time_window', 'date', 'hour', 'weekday']).size()\
       .reset_index().rename(columns = {0:'volume'})

In [6]:
pd_volume_train.head()

,time,tollgate_id,direction,time_window,date,hour,weekday,volume
0,2016-09-19,1,0,"[00:00:00,00:20:00)",2016-09-19,0,1,13
1,2016-09-19,1,1,"[00:00:00,00:20:00)",2016-09-19,0,1,140
2,2016-09-19,2,0,"[00:00:00,00:20:00)",2016-09-19,0,1,2
3,2016-09-19,3,0,"[00:00:00,00:20:00)",2016-09-19,0,1,17
4,2016-09-19,3,1,"[00:00:00,00:20:00)",2016-09-19,0,1,181


In [36]:
baseline = RandomForestRegressor()

In [37]:
baseline.get_params().keys()

dict_keys(['bootstrap', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [10]:
metricas = ['explained_variance', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'r2']

dict_keys(['constant', 'quantile', 'strategy'])

In [35]:
# Create the parameter grid based on the results of random search 
hyper = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

In [41]:
grid_search = GridSearchCV(estimator = baseline, param_grid = hyper, cv = 3, n_jobs = -1, verbose = 2)

In [42]:
def feature_format():
    v_train = pd.read_csv('dados_treino_volume_com_valor_anterior.csv')
    v_test = pd.read_csv('dados_teste_volume_com_valor_anterior.csv')
    #pd_volume_train = pd_volume_train.set_index(['time'])
    #pd_volume_test = pd_volume_test.set_index(['time'])
    #volume_train = v_train.groupby(['time_window','tollgate_id','direction','date', 'hour']).size().reset_index().rename(columns = {0:'volume'})
    #volume_test = v_test.groupby(['time_window','tollgate_id','direction','date', 'hour']).size().reset_index().rename(columns = {0:'volume'})
    #print(volume_train)                
    x = pd.Series(v_train['time_window'].unique())
    s = pd.Series(range(len(x)),index = x.values)
    v_train['window_n'] = v_train['time_window'].map(s)
    v_test['window_n'] = v_test['time_window'].map(s)
#        print vol_test.tail()
    #volume_train['weekday'] = v_train['weekday']
    #volume_test['weekday'] = v_test['weekday']
    
    feature_train = v_train.drop('volume', axis = 1)
    feature_test = v_test.drop('volume',axis = 1)
    values_train = v_train['volume'].values
    values_test = v_test['volume'].values
    
    return feature_train, feature_test, values_train, values_test

In [43]:
feature_train, feature_test, values_train, values_test = feature_format()

In [44]:
grid_search.fit(feature_train[['window_n','tollgate_id', 'direction', 'weekday', 'volume_anterior', 'volume_anterior_2','media_volume', 'desvio_padrao']], values_train)

Fitting 3 folds for each of 288 candidates, totalling 864 fits
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100 
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100 
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100 
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=200 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   1.1s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=200 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   1.1s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=200 
[CV]  bootstrap=True, max_d

[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=8, n_estimators=100 


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   38.2s


[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=8, n_estimators=100, total=   1.1s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=8, n_estimators=100, total=   1.1s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=8, n_estimators=100, total=   1.1s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=8, n_estimators=200 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=8, n_estimators=200, total=   2.3s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=8, n_estimators=200 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samp

[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=12, n_estimators=300, total=   3.6s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=5, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=5, min_samples_split=8, n_estimators=100, total=   1.1s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=5, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=5, min_samples_split=8, n_estimators=100, total=   1.1s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=5, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=5, min_samples_split=8, n_estimators=100, total=   1.1s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=5, min_samples_split=8, n_estimators=200 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_sam

...

[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=3, min_samples_split=12, n_estimators=100, total=   1.3s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=3, min_samples_split=12, n_estimators=100 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=3, min_samples_split=12, n_estimators=100, total=   1.4s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=3, min_samples_split=12, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=3, min_samples_split=12, n_estimators=200, total=   3.0s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=3, min_samples_split=12, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=3, min_samples_split=12, n_estimators=200, total=   2.6s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=3, min_samples_split=12, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_fea

[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=100 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=100, total=   1.4s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=100 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=100, total=   1.3s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=100 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=100, total=   1.4s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=200, total=   2.8s
[CV] bootstrap=True, max_depth=110, max_feat

[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=300, total=   3.9s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=1000 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=300, total=   4.1s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=100 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=100, total=   1.4s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=100 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=100, total=   1.3s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=100 
[CV]  bootstrap=True, max_depth=110, max_fe

[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed: 20.6min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'bootstrap': [True], 'max_depth': [80, 90, 100, 110], 'max_features': [2, 3], 'min_samples_leaf': [3, 4, 5], 'min_samples_split': [8, 10, 12], 'n_estimators': [100, 200, 300, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [46]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 110,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 100}

In [48]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [50]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, feature_test[['window_n','tollgate_id', 'direction', 'weekday', 'volume_anterior', 'volume_anterior_2','media_volume', 'desvio_padrao']], values_test)

Model Performance
Average Error: 10.9749 degrees.
Accuracy = 73.40%.


In [51]:
print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

NameError: name 'base_accuracy' is not defined

In [21]:
pd.set_option('max_columns',200)
pd.DataFrame(meu_primeiro_grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_constant,param_quantile,param_strategy,params,split0_test_explained_variance,split1_test_explained_variance,split2_test_explained_variance,mean_test_explained_variance,std_test_explained_variance,rank_test_explained_variance,split0_test_neg_mean_absolute_error,split1_test_neg_mean_absolute_error,split2_test_neg_mean_absolute_error,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error,rank_test_neg_mean_absolute_error,split0_test_neg_mean_squared_error,split1_test_neg_mean_squared_error,split2_test_neg_mean_squared_error,mean_test_neg_mean_squared_error,std_test_neg_mean_squared_error,rank_test_neg_mean_squared_error,split0_test_neg_mean_squared_log_error,split1_test_neg_mean_squared_log_error,split2_test_neg_mean_squared_log_error,mean_test_neg_mean_squared_log_error,std_test_neg_mean_squared_log_error,rank_test_neg_mean_squared_log_error,split0_test_neg_median_absolute_error,split1_test_neg_median_absolute_error,split2_test_neg_median_absolute_error,mean_test_neg_median_absolute_error,std_test_neg_median_absolute_error,rank_test_neg_median_absolute_error,split0_test_r2,split1_test_r2,split2_test_r2,mean_test_r2,std_test_r2,rank_test_r2
0,0.002389,0.000355,0.003711,0.000443,300000,0.75,mean,"{'constant': 300000, 'quantile': 0.75, 'strate...",0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,2,-35.790767,-48.177580,-34.678188,-39.548471,6.118131,1,-1.770421e+03,-3.221451e+03,-1.613624e+03,-2.201789e+03,7.237925e+02,1,-1.090409,-2.092558,-1.114232,-1.432366,0.466893,2,-35.513715,-44.112684,-35.461917,-38.362489,4.065754,2,-1.181346e-02,-1.227511e-02,-1.800480e-03,-8.630000e-03,4.832517e-03,1
1,0.003224,0.000209,0.003598,0.000285,300000,0.75,median,"{'constant': 300000, 'quantile': 0.75, 'strate...",0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,2,-37.657526,-45.718247,-35.895349,-39.756832,4.275998,2,-2.281523e+03,-3.183131e+03,-2.058676e+03,-2.507754e+03,4.861173e+02,2,-1.016549,-1.902855,-1.003700,-1.307672,0.420859,1,-28.000000,-39.000000,-27.000000,-31.333002,5.436334,1,-3.039138e-01,-2.337856e-04,-2.781061e-01,-1.940955e-01,1.374750e-01,2
2,0.002697,0.000151,0.002643,0.000161,300000,0.75,quantile,"{'constant': 300000, 'quantile': 0.75, 'strate...",0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,2,-43.681669,-62.712284,-44.232856,-50.208288,8.843865,3,-2.646141e+03,-4.714127e+03,-2.688869e+03,-3.349642e+03,9.649219e+02,3,-1.635530,-3.008534,-1.695379,-2.113100,0.633592,3,-43.000000,-71.000000,-44.000000,-52.665706,12.969764,3,-5.122966e-01,-4.813179e-01,-6.693545e-01,-5.543188e-01,8.231385e-02,3
3,0.001825,0.000080,0.002457,0.000177,300000,0.75,constant,"{'constant': 300000, 'quantile': 0.75, 'strate...",1.110223e-16,0.0,0.0,3.701479e-17,5.233902e-17,1,-299942.939791,-299950.137448,-299944.835122,-299945.970486,3.046184,4,-8.996577e+10,-8.997009e+10,-8.996691e+10,-8.996759e+10,1.828056e+06,4,-80.761385,-88.835583,-81.278027,-83.624714,3.690399,4,-299950.000000,-299979.000000,-299953.000000,-299960.665607,13.021137,4,-5.141635e+07,-2.827126e+07,-5.585495e+07,-4.518147e+07,1.209298e+07,4
